# Bootstrap Dataset
Download the food-101 dataset for the list of assigned categories,
saving the bootstrapped dataset in minio.

In [ ]:
import os, shutil

In [2]:
data_dir = f"{os.environ['HOME']}/data-vol-1"

#### Download &amp; Extract Dataset

In [3]:
!wget -O {data_dir}/food-101.tgz http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

--2020-11-30 06:39:41--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2020-11-30 06:39:45--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘/home/jovyan/data-vol-1/food-101.tgz’

/home/jovyan/data-v 100%[===================>]   4.65G  2.47MB/s    in 30m 13s 

2020-11-30 07:09:59 (2.63 MB/s) - ‘/home/jovyan/data-vol-1/food-101.tgz’ saved [4996278331/4996278331]



In [3]:
%%time
!tar xzf {data_dir}/food-101.tgz -C {data_dir}

CPU times: user 265 ms, sys: 28.8 ms, total: 294 ms
Wall time: 27.7 s


#### Extract Relevant Categories from Dataset

Obtain the list of relevant categories.


In [6]:
!unzip -f $HOME/assign_1/Food_list.zip -d /tmp/
!cp /tmp/Food_list/50.txt $HOME/assign_1/notebooks/50.txt

Archive:  /home/jovyan/assign_1/Food_list.zip


Extract the relevant food categories out from the dataset based on the categories assigned in list `50.txt`

In [7]:
image_dir = data_dir + "/food-101/images"
base_dir = data_dir + "/food-101/zzy"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [8]:
# Directories for your training, validation and test splits
shutil.rmtree(base_dir)
os.mkdir(base_dir)
os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)

In [9]:
# Assign the 10 types of food from your .txt file to a list variable 'food_list'
   
label_file = os.path.join('50.txt') 
# Refer the report Appendix
# Please enter the name of .txt file which contains a list of food assigned to you
# Make Sure you copy the .txt file in base_dir

with open(label_file, 'r') as f:
    x = f.readlines()
    
food_list =[]
for item in x:
    if item == '\n':
        continue        
    else:
        food_list.append(item.strip('\n'))

In [10]:
#copy the first 750 images to train folder
for item in food_list:
    train_food_dir = os.path.join(train_dir, item)
    os.mkdir(train_food_dir)
    img_list = os.listdir(os.path.join(image_dir, item))[:750]
    for fname in img_list:
        src = os.path.join(image_dir, item, fname)
        dst = os.path.join(train_food_dir, fname)
        shutil.copyfile(src, dst)

In [11]:
#copy the following 200 images [750:950] to validation folder
for item in food_list:
    validation_food_dir = os.path.join(validation_dir, item)
    os.mkdir(validation_food_dir)
    img_list = os.listdir(os.path.join(image_dir, item))[750:950]
    for fname in img_list:
        src = os.path.join(image_dir, item, fname)
        dst = os.path.join(validation_food_dir, fname)
        shutil.copyfile(src, dst)

In [12]:
#copy the rest 50 images [950:1000] to test folder
for item in food_list:
    test_food_dir = os.path.join(test_dir, item)
    os.mkdir(test_food_dir)
    img_list = os.listdir(os.path.join(image_dir, item))[950:1000]
    for fname in img_list:
        src = os.path.join(image_dir, item, fname)
        dst = os.path.join(test_food_dir, fname)
        shutil.copyfile(src, dst)

#### Upload Bootstrapped Dataset 
Upload bootstrapped dataset to minio for later use.

In [27]:
from minio import Minio
minio = Minio(os.environ["MINIO_HOST"],
              access_key=os.environ["MINIO_ACCESS_KEY"],
              secret_key=os.environ["MINIO_SECRET_KEY"],
              secure=False)

{'name': 'datasets', 'creation_date': datetime.datetime(2020, 12, 1, 8, 0, 59, 179000, tzinfo=<UTC>)}


In [29]:
data_bucket = "datasets"
if data_bucket not in [ b.name for b in minio.list_buckets() ]:
    minio.make_bucket(data_bucket)

In [43]:
for root, _, files in os.walk(base_dir):
    # strip the leading directory
    dir_path = root.replace(base_dir, "")
    imgs_files = [ f for f in files if f[-3:] == "jpg" ]
    for img_path in imgs_files:
        minio.fput_object(bucket_name=data_bucket,
                          object_name=f"food-101{dir_path}/{img_path}",
                          file_path=os.path.join(root, img_path),
                          content_type="image/jpeg")
                          